In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# Import pandas
import pandas as pd

# Create the DataFrame
data = {
    'File': [f'{i}.mp4' for i in range(1, 112)],
    'Senti': [1] * 111
}

df = pd.DataFrame(data)

# Save to CSV in Google Drive
file_path = '/content/drive/MyDrive/work2/Meta_Data/Videos.csv'
df.to_csv(file_path, index=False)

print(f"File saved to {file_path}")

File saved to /content/drive/MyDrive/work2/Meta_Data/Videos.csv


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/work2/Validation_Data/key_positions_result.csv')

# Copy the 'File' column
df_new = df[['File']].copy()

# Add the 'Senti' column with value 1 for each entry
df_new['Senti'] = 1

# Save the new dataframe to the specified path
output_path = '/content/drive/MyDrive/work2/Validation_Data/Meta_Data/Validation_Video.csv'
df_new.to_csv(output_path, index=False)

df_new.head()


,File,Senti
0,1. Mobile_Neutral.wav,1
1,1. Mobile__Positive_Product_Review.wav,1
2,1. Tea_Negative_Product_Review.wav,1
3,10. Blasphemy_Negative_Perception_Building.wav,1
4,10. Political Extremisim_Negative_Perception_B...,1


In [ ]:
 import pandas as pd

# Load the updated transcriptions with sentiment file
transcriptions_path = '/content/drive/MyDrive/work2/updated_transcriptions_with_senti.csv'
t = pd.read_csv(transcriptions_path)

# Preserve original file names
t['Original_File'] = t['File']

# Extract numeric part from 'File' column
t['File'] = t['File'].str.extract('(\d+)', expand=False)

# Drop rows with NaN values in 'File' column
t = t.dropna(subset=['File'])

# Convert 'File' column to integers
t['File'] = t['File'].astype(int)

# Process testing data: filter rows where 'File' column values are between 79 and 111 (inclusive)
filtered_rows_testing = t[(t['File'] >= 79) & (t['File'] <= 111)]
selected_columns_testing = filtered_rows_testing[['Original_File', 'Transcription', 'Senti']].rename(columns={'Original_File':'File','Transcription': 'Sentence'})
testing_text_csv_path = '/content/drive/MyDrive/work2/Testing_Data/test_text/Text.csv'
selected_columns_testing.to_csv(testing_text_csv_path, index=False)
print(f"Testing data file created and saved to {testing_text_csv_path}")

# Process training data: filter rows where 'File' column values are less than 79
filtered_rows_training = t[t['File'] < 79]
selected_columns_training = filtered_rows_training[['Original_File', 'Transcription', 'Senti']].rename(columns={'Original_File':'File','Transcription': 'Sentence'})
training_text_csv_path = '/content/drive/MyDrive/work2/Training_Data/train_text/Text.csv'
selected_columns_training.to_csv(training_text_csv_path, index=False)
print(f"Training data file created and saved to {training_text_csv_path}")



Updated file saved to /content/drive/MyDrive/work2/Testing_Data/test_text/Text.csv


In [ ]:
# prompt: load this and show columns names /content/drive/MyDrive/work2/Meta_Data/Frames.csv'

import pandas as pd

# Load the CSV file from Google Drive
file_path = '/content/drive/MyDrive/work2/Meta_Data/Frames.csv'
df = pd.read_csv(file_path)

# Get the column names
column_names = list(df.columns)

# Print the column names
print(column_names)


['Video Name', 'Frame Path', 'Time in Seconds', 'Key_Frame', 'Positive Sentiment', 'Negative Sentiment', 'key_frame']


In [ ]:
# prompt: load this file '/content/drive/MyDrive/work2/Meta_Data/Frames.csv'   check if 'Positive Sentiment', 'Negative Sentiment' values is greater than 1.0
# Also sort on 'Positive Sentiment', 'Negative Sentiment'
# Also give count of each group on 'Positive Sentiment', 'Negative Sentiment'

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/work2/Meta_Data/Audio.csv')

# Filter rows where 'Positive Sentiment' and 'Negative Sentiment' values are greater than 1.0
#filtered_df = df[(df['Positive Sentiment'] > 0.0) | (df['Negative Sentiment'] > 0.0)]

# Sort the filtered DataFrame by 'Positive Sentiment' and 'Negative Sentiment'
sorted_df = df.sort_values(by=['Senti'])

# Group the sorted DataFrame by 'Positive Sentiment' and 'Negative Sentiment' and count the number of rows in each group
grouped_df = sorted_df.groupby(['Senti']).size().to_frame(name='Count')

# Print the grouped DataFrame
grouped_df.head(50)


,Count
Senti,
0,17
1,54
2,40


In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# Load the data from the new file path
df = pd.read_csv('/content/drive/MyDrive/work2/Meta_Data/Frames_test.csv')

# 1. Replace all values of 9.0 with 0.0 in both 'Positive Sentiment' and 'Negative Sentiment' columns
df['Positive Sentiment'] = df['Positive Sentiment'].replace(9.0, 0.0)
df['Negative Sentiment'] = df['Negative Sentiment'].replace(9.0, 0.0)

# Handle non-finite values by replacing NA and inf with 0.0
df['Positive Sentiment'].replace([np.inf, -np.inf], np.nan, inplace=True)
df['Negative Sentiment'].replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0.0, inplace=True)

# 2. Add a new column 'Senti' with data type 'int' and copy the greater values from 'Positive Sentiment' and 'Negative Sentiment'
df['Senti'] = df[['Positive Sentiment', 'Negative Sentiment']].max(axis=1).astype(int)

# 3. Change values of 'Senti' where 'Key_Frame' is 'N' according to the given data distribution
# Filter rows where 'Key_Frame' is 'N'
key_frame_n = df[df['Key_Frame'] == 'N']

# Calculate the number of rows to assign for each value based on the given percentages
total_n_rows = len(key_frame_n)
num_zeros = int(total_n_rows * 0.15)
num_ones = int(total_n_rows * 0.50)
num_twos = total_n_rows - num_zeros - num_ones  # Remaining rows

# Create new values for 'Senti' based on the required distribution
new_senti_values = [0] * num_zeros + [1] * num_ones + [2] * num_twos

# Shuffle the new 'Senti' values to distribute them randomly
np.random.shuffle(new_senti_values)

# Assign the new values back to the DataFrame
df.loc[df['Key_Frame'] == 'N', 'Senti'] = new_senti_values

# Save the updated DataFrame to a new CSV file
df.to_csv('/content/drive/MyDrive/work2/Meta_Data/Frames_test_updated.csv', index=False)

# Show distribution of 'Senti'
senti_distribution = df['Senti'].value_counts().to_frame(name='Count')

# Group by 'Senti' and show all columns
grouped_df = df.groupby('Senti').apply(lambda x: x.describe()).unstack()

# Display the results
print(senti_distribution)
print(grouped_df.head(50))


       Count
Senti       
0       1218
1       1127
2        788
      Video Name                                                      \
           count       mean        std  min   25%   50%   75%    max   
Senti                                                                  
0         1218.0  49.788998  33.987101  1.0  19.0  49.0  80.0  111.0   
1         1127.0  58.709849  28.945716  1.0  35.0  59.0  82.0  111.0   
2          788.0  60.184010  28.626995  1.0  36.0  62.0  81.0  111.0   

      Time in Seconds             ... Negative Sentiment        Senti       \
                count       mean  ...                75%  max   count mean   
Senti                             ...                                        
0              1218.0  16.144499  ...                0.0  2.0  1218.0  0.0   
1              1127.0  14.343390  ...                0.0  2.0  1127.0  1.0   
2               788.0  14.823604  ...                0.0  2.0   788.0  2.0   

                                 

In [ ]:
print(senti_distribution)

       Count
Senti       
0       1218
1       1127
2        788


In [ ]:
df.head()

,Video Name,Frame Path,Time in Seconds,Key_Frame,Positive Sentiment,Negative Sentiment,key_frame,Senti
0,46,/content/drive/MyDrive/work2/Frames/46/46_fram...,0,N,0.0,0.0,Y,2
1,46,/content/drive/MyDrive/work2/Frames/46/46_fram...,1,N,0.0,0.0,Y,1
2,46,/content/drive/MyDrive/work2/Frames/46/46_fram...,2,N,0.0,0.0,Y,1
3,46,/content/drive/MyDrive/work2/Frames/46/46_fram...,3,Y,0.0,0.0,Y,0
4,46,/content/drive/MyDrive/work2/Frames/46/46_fram...,4,N,0.0,0.0,Y,0


In [ ]:
print(grouped_df.head(50))

      Video Name                                                      \
           count       mean        std  min   25%   50%   75%    max   
Senti                                                                  
0         1218.0  49.788998  33.987101  1.0  19.0  49.0  80.0  111.0   
1         1127.0  58.709849  28.945716  1.0  35.0  59.0  82.0  111.0   
2          788.0  60.184010  28.626995  1.0  36.0  62.0  81.0  111.0   

      Time in Seconds             ... Negative Sentiment        Senti       \
                count       mean  ...                75%  max   count mean   
Senti                             ...                                        
0              1218.0  16.144499  ...                0.0  2.0  1218.0  0.0   
1              1127.0  14.343390  ...                0.0  2.0  1127.0  1.0   
2               788.0  14.823604  ...                0.0  2.0   788.0  2.0   

                                     
       std  min  25%  50%  75%  max  
Senti                 

In [ ]:
import pandas as pd

# Assuming df is your DataFrame after applying the changes

# Convert 'Senti' column to categorical type
df['Senti'] = pd.Categorical(df['Senti'], categories=[0, 1, 2], ordered=True)

# Find the distribution of each category in the 'Senti' column
category_distribution = df['Senti'].value_counts()

# Display the distribution
print(category_distribution)


Senti
0    899
1     63
2     43
Name: count, dtype: int64


In [2]:
import pandas as pd

# Load the CSV file
csv_path = '/content/drive/MyDrive/work2/Validation_Data/Meta_Data/Frames_Meta_Data.csv'
df = pd.read_csv(csv_path)

# Function to update Video Name based on Frame Path
def update_video_name(row):
    # Extract the part of the Frame Path to construct the new Video Name
    path_parts = row['Frame Path'].split('/')
    video_name = path_parts[-2].replace('_frame_', '.wav')
    return video_name

# Update the Video Name column
df['Video Name'] = df.apply(update_video_name, axis=1)

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_path, index=False)

print(f"Updated Video Name column and saved the results to {csv_path}")


Updated Video Name column and saved the results to /content/drive/MyDrive/work2/Validation_Data/Meta_Data/Frames_Meta_Data.csv


In [3]:
import pandas as pd

# Load the CSV file
csv_path = '/content/drive/MyDrive/work2/Validation_Data/Meta_Data/Frames_Meta_Data.csv'
df = pd.read_csv(csv_path)

# Function to update Video Name based on Frame Path
def update_video_name(row):
    # Extract the relevant part of the Frame Path to construct the new Video Name
    path_parts = row['Frame Path'].split('/')
    video_name_part = path_parts[-2].split('_frame_')[0]
    return video_name_part + '.wav'

# Update the Video Name column
df['Video Name'] = df.apply(update_video_name, axis=1)

# Save the updated DataFrame back to the CSV file
df.to_csv(csv_path, index=False)

print(f"Updated Video Name column and saved the results to {csv_path}")


Updated Video Name column and saved the results to /content/drive/MyDrive/work2/Validation_Data/Meta_Data/Frames_Meta_Data.csv
